In [1]:
import numpy as np
import pandas as pd

# Отключение некоторых лишних предупреждений
import warnings
warnings.filterwarnings("ignore")

## Часть 2: Cоздание новых признаков

_Над этой частью работали Халифаева Лейла и Рыбьякова Лиза_

### 0. Выгрузка данных

In [2]:
data_stroke= pd.read_csv('data_stroke.csv')
data_heart= pd.read_csv('data_heart.csv')

### 1. Создание переменных

1) В дататасетах есть информация об Индексе массы тела, поэтому мы решили создать в каждом из них новую переменную, которая будет отражать, является ли вес респондента избыточным(1) или нет(0), то есть превышает ли норму в 25 единиц или нет.

In [3]:
data_heart['BMI_excess'] = data_heart['BMI']
data_heart['BMI_excess'] = np.where((data_heart['BMI'] < 25), 0, data_heart['BMI_excess'])
data_heart['BMI_excess'] = np.where((data_heart['BMI'] >= 25), 1, data_heart['BMI_excess'])

In [4]:
data_stroke['BMI_excess'] = data_stroke['bmi']
data_stroke['BMI_excess'] = np.where((data_stroke['bmi'] < 25), 0, data_stroke['BMI_excess'])
data_stroke['BMI_excess'] = np.where((data_stroke['bmi'] >= 25), 1, data_stroke['BMI_excess'])

2) Также в данных об инсультах есть переменная, отражающая средний уровень глюкозы в крови респондента. Создадим пременную, которая покажет, является ли показатель респондента нормальным(1) или нет(0). Нормой является уровень в пределах от 80 до 115 мг/дл.

In [5]:
data_stroke['glucose_level_normal'] = data_stroke['avg_glucose_level']
data_stroke['glucose_level_normal'] = np.where(((data_stroke['avg_glucose_level'] >= 80) & 
                                                (data_stroke['avg_glucose_level'] <= 115)), 1,
                                                 data_stroke['glucose_level_normal'])
data_stroke['glucose_level_normal'] = np.where(((data_stroke['avg_glucose_level'] < 80) | 
                                                (data_stroke['avg_glucose_level'] > 115)), 0,
                                                 data_stroke['glucose_level_normal'])

3) В первом датасете есть много признаков, которые говорят в целом о состоянии здоровья и образе жизни респондентов, поэтому можно создать еще один признак. 

_Предположим, что человек, который ведет здоровый образ жизни, соответствует следующим критериям:_
- Не курит;
- Употребляет достаточное количество фруктов и овощей в пищу;
- Регулярно является физически активным в течение месяца;
- Не употребляют алкоголь;

In [6]:
data_heart['healthy_lifestyle'] = np.zeros(data_heart.shape[0])
data_heart['healthy_lifestyle'] = np.where(((data_heart['Smoker'] == 0) &
                                            (data_heart['Fruits'] == 1) &
                                            (data_heart['Veggies'] == 1) &
                                            (data_heart['PhysActivity'] == 1) &
                                            (data_heart['HvyAlcoholConsump'] == 0)),
                                            1, data_heart['healthy_lifestyle'])

### Сохраним преобразованные данные:

In [7]:
data_stroke.to_csv('data_stroke.csv', index=False)
data_heart.to_csv('data_heart.csv', index=False)